# 📊 BACKTEST SIMPLE v0.2.0

Testing de PASO 1-5 sin interferir con live trading.

**Flujo:**
1. Carga histórico (BTC/ETH últimos 6 meses)
2. Simula Paso 1-5 en cada vela
3. Calcula Sortino, Win Rate, Profit Factor
4. Valida cambios ANTES de deploy

In [ ]:
import pandas as pd
import numpy as np
import yaml
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Config
with open('config/config.yaml') as f:
    config = yaml.safe_load(f)

print("✅ Backtest environment initialized")

## 1️⃣ CARGAR HISTÓRICO

In [ ]:
# Carga datos históricos de archivo CSV
# En producción, usar scraper de exchange

def load_backtest_data(asset: str, csv_path: str = None) -> pd.DataFrame:
    """Carga datos históricos para backtest"""
    try:
        if csv_path is None:
            csv_path = f"{asset}_USD_data.csv"
        
        df = pd.read_csv(csv_path, index_col=0, parse_dates=True)
        
        # Limpiar
        df = df.dropna()
        df = df[['Close', 'Volume']].rename(columns={'Close': 'price', 'Volume': 'volume'})
        
        # Features
        df['returns'] = df['price'].pct_change()
        df['log_returns'] = np.log(df['price'] / df['price'].shift(1))
        df['volatility'] = df['returns'].rolling(20).std()
        df['ma_20'] = df['price'].rolling(20).mean()
        df['ma_50'] = df['price'].rolling(50).mean()
        
        df = df.dropna()
        
        print(f"✅ Loaded {asset}: {len(df)} candles, {df.index[0].date()} to {df.index[-1].date()}")
        return df
    except Exception as e:
        print(f"❌ Error loading {asset}: {e}")
        return None

# Cargar assets
btc_df = load_backtest_data('BTC')
eth_df = load_backtest_data('ETH')

print(f"\n📊 Datasets:")
print(f"  BTC: {len(btc_df)} candles" if btc_df is not None else "  BTC: No data")
print(f"  ETH: {len(eth_df)} candles" if eth_df is not None else "  ETH: No data")

## 2️⃣ SIMULACIÓN DE PASO 1-5

In [ ]:
class SimpleBacktester:
    """Simulador simple del sistema (Paso 1-5)"""
    
    def __init__(self, initial_capital: float = 100000):
        self.initial_capital = initial_capital
        self.capital = initial_capital
        self.trades = []
        self.positions = {}
    
    def generate_signal(self, row: pd.Series, asset: str) -> str:
        """
        Genera signal basado en simple heurística
        En producción: usa 9 expertos
        """
        
        # Signal: LONG si MA20 > MA50 y volatilidad baja
        if row['ma_20'] > row['ma_50'] and row['volatility'] < row['volatility'].rolling(50).mean():
            return 'LONG'
        
        # SHORT si MA20 < MA50
        elif row['ma_20'] < row['ma_50']:
            return 'SHORT'
        
        else:
            return 'HOLD'
    
    def calculate_position_size(self, confidence: float = 0.7) -> float:
        """Calcula % del capital a arriesgar (Kelly Criterion simplificado)"""
        return self.capital * 0.05 * confidence  # 5% base * confidence
    
    def execute_trade(self, asset: str, signal: str, price: float, timestamp):
        """Ejecuta trade simulado"""
        
        # Posición existente
        if asset in self.positions:
            exit_trade = self.positions[asset]
            
            # Cerrar
            pnl = (price - exit_trade['entry_price']) * exit_trade['quantity']
            if exit_trade['side'] == 'SHORT':
                pnl *= -1
            
            self.capital += pnl
            
            self.trades.append({
                'asset': asset,
                'entry_price': exit_trade['entry_price'],
                'exit_price': price,
                'quantity': exit_trade['quantity'],
                'side': exit_trade['side'],
                'pnl': pnl,
                'pnl_pct': pnl / (exit_trade['entry_price'] * exit_trade['quantity']),
                'entry_time': exit_trade['entry_time'],
                'exit_time': timestamp
            })
            
            del self.positions[asset]
        
        # Nueva posición (si signal)
        if signal != 'HOLD':
            position_size = self.calculate_position_size()
            quantity = position_size / price
            
            # Stop loss
            stop_loss = price * 0.98 if signal == 'LONG' else price * 1.02
            
            self.positions[asset] = {
                'side': signal,
                'entry_price': price,
                'quantity': quantity,
                'stop_loss': stop_loss,
                'entry_time': timestamp
            }
    
    def run_backtest(self, data_dict: dict, max_trades: int = 50):
        """Ejecuta backtest"""
        
        trade_count = 0
        
        # Iterar cronológicamente
        for idx in range(100, len(next(iter(data_dict.values())))):
            if trade_count >= max_trades:
                break
            
            for asset, df in data_dict.items():
                if idx >= len(df):
                    continue
                
                row = df.iloc[idx]
                timestamp = df.index[idx]
                
                # Generar signal
                signal = self.generate_signal(df.iloc[max(0, idx-50):idx+1], asset)
                
                # Ejecutar
                self.execute_trade(asset, signal, row['price'], timestamp)
                trade_count += len([t for t in self.trades if t not in self.trades[:-1]])
    
    def get_metrics(self) -> dict:
        """Calcula métricas finales"""
        
        if len(self.trades) == 0:
            return {}
        
        df_trades = pd.DataFrame(self.trades)
        
        returns = df_trades['pnl_pct'].values
        
        # Sortino Ratio
        downside_returns = returns[returns < 0]
        if len(downside_returns) > 0:
            downside_dev = np.std(downside_returns)
            sortino = np.mean(returns) / downside_dev if downside_dev > 0 else 0
        else:
            sortino = np.mean(returns) * 100
        
        # Win Rate
        win_rate = np.sum(returns > 0) / len(returns)
        
        # Profit Factor
        wins = df_trades[df_trades['pnl'] > 0]['pnl'].sum()
        losses = abs(df_trades[df_trades['pnl'] < 0]['pnl'].sum())
        profit_factor = wins / losses if losses > 0 else (wins / 0.01 if wins > 0 else 0)
        
        # Drawdown
        cumulative = np.cumprod(1 + returns)
        running_max = np.maximum.accumulate(cumulative)
        drawdown = (cumulative - running_max) / running_max
        max_dd = np.min(drawdown)
        
        return {
            'total_trades': len(self.trades),
            'total_pnl': df_trades['pnl'].sum(),
            'total_pnl_pct': (self.capital - self.initial_capital) / self.initial_capital,
            'win_rate': win_rate,
            'sortino_ratio': sortino,
            'profit_factor': profit_factor,
            'max_drawdown': max_dd,
            'avg_trade_pnl': df_trades['pnl'].mean(),
            'capital_final': self.capital
        }


# Ejecutar backtest
print("\n🚀 Ejecutando backtest...\n")

backtester = SimpleBacktester(initial_capital=100000)

data_dict = {'BTC': btc_df, 'ETH': eth_df}
data_dict = {k: v for k, v in data_dict.items() if v is not None}

backtester.run_backtest(data_dict, max_trades=50)

metrics = backtester.get_metrics()

print("\n" + "="*70)
print("📊 BACKTEST RESULTS")
print("="*70)
print(f"""Total Trades:     {metrics.get('total_trades', 0):>10}
Total PnL:        {metrics.get('total_pnl', 0):>10,.2f} USDT
Total Return:     {metrics.get('total_pnl_pct', 0):>10.2%}
Win Rate:         {metrics.get('win_rate', 0):>10.2%}
Sortino Ratio:    {metrics.get('sortino_ratio', 0):>10.2f}
Profit Factor:    {metrics.get('profit_factor', 0):>10.2f}
Max Drawdown:     {metrics.get('max_drawdown', 0):>10.2%}
Avg Trade PnL:    {metrics.get('avg_trade_pnl', 0):>10,.2f}
Final Capital:    {metrics.get('capital_final', 0):>10,.2f} USDT""")
print("="*70)

## 3️⃣ VALIDACIÓN

In [ ]:
# Criterios de validación
print("\n✅ VALIDACIÓN PRE-DEPLOYMENT:\n")

checks = [
    ("Sortino > 1.5", metrics.get('sortino_ratio', 0) > 1.5),
    ("Win Rate > 50%", metrics.get('win_rate', 0) > 0.50),
    ("Profit Factor > 1.5", metrics.get('profit_factor', 0) > 1.5),
    ("Max Drawdown < 20%", metrics.get('max_drawdown', 0) > -0.20),
    ("Mín 20 trades", metrics.get('total_trades', 0) >= 20),
]

passed = 0
for check_name, result in checks:
    status = "✅ PASS" if result else "❌ FAIL"
    print(f"  {status} - {check_name}")
    if result:
        passed += 1

print(f"\n{'🟢 LISTO PARA DEPLOY' if passed >= 4 else '🔴 AJUSTA PARÁMETROS'} ({passed}/5 checks)")

## 4️⃣ HISTÓRICO DE TRADES

In [ ]:
# Mostrar últimos trades
if backtester.trades:
    df_trades = pd.DataFrame(backtester.trades[-10:])  # Últimos 10
    print("\n📈 Últimos Trades:")
    print(df_trades[['asset', 'side', 'entry_price', 'exit_price', 'pnl', 'pnl_pct']].to_string(index=False))
else:
    print("\nSin trades ejecutados")